#  Exploration des articles de 1920 (thème : immigration de la prese belge)

Dans cette étape, j’explore le sous-ensemble 1920 du corpus CAMille pour repérer un article pertinent sur l’immigration. Je vérifie le chemin, je compte les fichiers, j’applique un filtre par mots-clés, puis je regarde des extraits avant de choisir un article et d’enregistrer sa référence.

# 1) Définir le chemin vers les fichiers et vérifier l’accès
Je pointe `data_path` vers les TXT et je teste que le dossier existe pour éviter une erreur de chemin avant la suite.

In [ ]:

import os

data_path = "../data/txt"  # adapte si besoin
assert os.path.isdir(data_path), f"Dossier introuvable : {data_path}"

# 2) Lister les fichiers de l’année 1920 et compter
Je récupère tous les `.txt` qui contiennent “1920” dans le nom pour savoir combien de documents je vais explorer (et j’affiche quelques exemples).

In [ ]:
# Lister les fichiers de l'année 1920
files_1920 = sorted([f for f in os.listdir(data_path) if "1920" in f and f.endswith(".txt")])
print("Nombre de fichiers 1920 :", len(files_1920))
files_1920[:10]

Nombre de fichiers 1920 : 100


['KB_JB838_1920-01-13_01-00005.txt',
 'KB_JB838_1920-01-18_01-00014.txt',
 'KB_JB838_1920-02-04_01-00001.txt',
 'KB_JB838_1920-02-04_01-00002.txt',
 'KB_JB838_1920-02-06_01-00004.txt',
 'KB_JB838_1920-02-07_01-00006.txt',
 'KB_JB838_1920-02-08_01-00005.txt',
 'KB_JB838_1920-02-08_01-00010.txt',
 'KB_JB838_1920-02-10_01-00002.txt',
 'KB_JB838_1920-02-27_01-00011.txt']

# 3) Filtrer les articles 1920 liés à l’immigration (mots-clés simples)
J’applique un filtre lexical (immigration, immigré·s, étranger, réfugié, asile, naturalisation, etc.). Je passe le texte en minuscules pour une recherche simple et je garde les fichiers qui matchent au moins un mot-clé.

In [11]:
# Filtrer les articles 1920 liés à l'immigration (mots-clés simples) 
keywords = [
    "immigration", "immigrant", "immigrants",
    "étranger", "étrangère", "étrangers",
    "réfugié", "réfugiés", "asile", "naturalisation", "immigration", "immigrant", "immigrants", "étranger", "étrangère", "étrangers",
    "réfugié", "réfugiés", "asile", "naturalisation",
    "séjour", "permis de séjour", "titre de séjour", "carte d’identité d’étranger",
    "visa", "passeport", "laissez-passer", "frontière", "douanes", "contrôle",
    "expulsion", "refoulement", "internement",
    "administration des étrangers", "police des étrangers", "office des réfugiés",
    "consulat", "ministère de l’intérieur",
    "apatride", "demandeur d’asile",
    "rapatriement", "déplacés", "secours", "réinstallation",
    "main-d’œuvre étrangère", "recrutement", "logement", "ouvriers étrangers"
]

articles_retenus = []
for fname in files_1920:
    path = os.path.join(data_path, fname)
    with open(path, encoding="utf-8") as f:
        txt = f.read().lower()
    if any(k in txt for k in keywords):
        articles_retenus.append(fname)

print("Nombre d'articles potentiels :", len(articles_retenus))
articles_retenus[:10]

Nombre d'articles potentiels : 86


['KB_JB838_1920-01-18_01-00014.txt',
 'KB_JB838_1920-02-04_01-00001.txt',
 'KB_JB838_1920-02-04_01-00002.txt',
 'KB_JB838_1920-02-06_01-00004.txt',
 'KB_JB838_1920-02-07_01-00006.txt',
 'KB_JB838_1920-02-08_01-00005.txt',
 'KB_JB838_1920-02-08_01-00010.txt',
 'KB_JB838_1920-02-10_01-00002.txt',
 'KB_JB838_1920-03-01_01-00004.txt',
 'KB_JB838_1920-03-04_01-00008.txt']

## 4) Aperçu rapide de 3 fichiers (≈1500 caractères)
Je lis un court extrait de trois fichiers retenus pour vérifier que le thème correspond vraiment (je garde en tête le bruit possible dû à l’OCR).

In [12]:
# Aperçu des 3 premiers candidats (extrait 1500 caractères) 
for fname in articles_retenus[:3]:
    p = os.path.join(data_path, fname)
    with open(p, encoding="utf-8") as f:
        t = f.read()
    print(f"\n=== APERÇU : {fname} ===")
    print(t[:1500])


=== APERÇU : KB_JB838_1920-01-18_01-00014.txt ===
Vos enfants grandiront, fortifieront î: et passeront an bon hiver :: Si vous leur donne* 9 fi jï/a âatsstf Éf IL âf‘ MS& À base d'huile de foie de morue et d'hypophosphiteâ solubles :rs: 0 fpm d Mit h i* I > 5 2 b &« «ta :ï il 1>‘3 <0 gmih VÎ ii J a ï j En vento è ta PHARMACIE ORIPEKOVC:», 07-30, flSai'ohè-aiJx-Poulats, artutelle*. j On peut écrira, téléphoner- (tri Brusotlsa 0245) ou a’adressoe ilfrectomant à i’otïi- j cîns. fîcmiso à riomtoilô gi’ctull» «sans toute S’agglomératlon. Envol repicie on prov. £§5 Dépôt sptciaiités Ui’ pfkovLü potr os'.oiuio et la région. l’iru-maoid Do jj-g Vriesse, 15, place o’Arrnes^ OMontle. 940 Mi Lj.sa.c3-i es «le X 9 IdJis^orajo. g?, o OASTRITUS, GASTRALGIES, PLJUITTïS, Oï.AlKHS, DYSPEPSIE, AIGREURS, aoîiiLE.MEN'is «t Bnm.unFS dt-stom\c, vo.\tib$EMekys verveux db gros. SE3SU. MAL f)E MH U, DIARRHEES PAH ATOME, etc. RADJCÀLKiVSFNT GUERIS PAR LE Dépôt general : G. CUVIER, ptaîcien a Rycîiefcrl-siir-li

## 5) Choisir l’article d’étude (avec un petit contrôle)
Je sélectionne **un** articles parmi les retenus (critère : le plus informatif/central pour mon thème) et je vérifie que le fichier existe bien avant d’en lire un plus long extrait (≈3000 caractères).

In [13]:
# Choisir l'article d'étude 

idx = 0
chosen_article = articles_retenus[idx]
full_path = os.path.join(data_path, chosen_article)

assert os.path.exists(full_path), f"Fichier introuvable : {full_path}"
print("Article choisi :", chosen_article)

with open(full_path, encoding="utf-8") as f:
    text = f.read()

print("\n=== EXTRAIT (3000 car.) ===\n")
print(text[:3000])

Article choisi : KB_JB838_1920-01-18_01-00014.txt

=== EXTRAIT (3000 car.) ===

Vos enfants grandiront, fortifieront î: et passeront an bon hiver :: Si vous leur donne* 9 fi jï/a âatsstf Éf IL âf‘ MS& À base d'huile de foie de morue et d'hypophosphiteâ solubles :rs: 0 fpm d Mit h i* I > 5 2 b &« «ta :ï il 1>‘3 <0 gmih VÎ ii J a ï j En vento è ta PHARMACIE ORIPEKOVC:», 07-30, flSai'ohè-aiJx-Poulats, artutelle*. j On peut écrira, téléphoner- (tri Brusotlsa 0245) ou a’adressoe ilfrectomant à i’otïi- j cîns. fîcmiso à riomtoilô gi’ctull» «sans toute S’agglomératlon. Envol repicie on prov. £§5 Dépôt sptciaiités Ui’ pfkovLü potr os'.oiuio et la région. l’iru-maoid Do jj-g Vriesse, 15, place o’Arrnes^ OMontle. 940 Mi Lj.sa.c3-i es «le X 9 IdJis^orajo. g?, o OASTRITUS, GASTRALGIES, PLJUITTïS, Oï.AlKHS, DYSPEPSIE, AIGREURS, aoîiiLE.MEN'is «t Bnm.unFS dt-stom\c, vo.\tib$EMekys verveux db gros. SE3SU. MAL f)E MH U, DIARRHEES PAH ATOME, etc. RADJCÀLKiVSFNT GUERIS PAR LE Dépôt general : G. CUVIER, 

## 6) Sauvegarder la référence de l’article choisi
Je stockke le nom du fichier sélectionné dans `data/article_selection.txt` pour le réutiliser facilement dans S3/S4.

In [14]:
# Sauvegarder la référence de l'article retenu 
out_ref = "../data/article_selectionne.txt"
with open(out_ref, "w", encoding="utf-8") as f:
    f.write(chosen_article + "\n")

print("Référence sauvegardée dans", out_ref)

Référence sauvegardée dans ../data/article_selectionne.txt
